In [52]:
import pandas as pd
import os
"""output 파일 양식 수정"""
folder_path = r"C:\VISSIM_Workspace\network_new\test"
file_name = sorted([f for f in os.listdir(folder_path) if f.endswith("output")])
excel_path = os.path.join(folder_path, file_name[0])
excel_files = sorted([f for f in os.listdir(excel_path) if f.endswith(".xlsx")], key=lambda x: int(x.split("_")[0]))

df = pd.read_excel(os.path.join(excel_path, excel_files[0]))
df["NEW"] = df["DATACOLLECTIONMEASUREMENT"] % 1000
df_grouped = df.groupby("DATACOLLECTIONMEASUREMENT").mean()
df_newGrouped = df_grouped.groupby("NEW").agg(lambda x: x.sum() if x.name == "VEHS(ALL)" else x.mean()).reset_index()
df = df_newGrouped[["NEW", "VEHS(ALL)", "QUEUEDELAY(ALL)"]]
del df_newGrouped

# 진입부
start_df = df[(df["NEW"] >= 1) & (df["NEW"] <=20)]
df1 = start_df.mean().to_frame().T

# 본선부
mid_df = df[(df["NEW"] >= 21) & (df["NEW"] <=100)]
df2 = mid_df.mean().to_frame().T

# 진출부
end_df = df[(df["NEW"] >= 101) & (df["NEW"] <=120)]
df3 = end_df.mean().to_frame().T

print(df1)
print("===============================================================")
print(df2)
print("===============================================================")
print(df3)
print("===============================================================")
df4 = pd.concat([df1, df2])
print(df4)

    NEW  VEHS(ALL)  QUEUEDELAY(ALL)
0  10.5  23.311167              0.0
    NEW  VEHS(ALL)  QUEUEDELAY(ALL)
0  60.5  23.278458              0.0
     NEW  VEHS(ALL)  QUEUEDELAY(ALL)
0  110.5  22.119417              0.0
    NEW  VEHS(ALL)  QUEUEDELAY(ALL)
0  10.5  23.311167              0.0
0  60.5  23.278458              0.0
